##IPerl Notebook for the HackaMol::Molecule class in Support of: 
###HackaMol: an object-oriented Modern Perl library for molecular hacking on multiple scales
Demian Riccardi, Jerry M. Parks, Alex Johs, and Jeremy C. Smith



####Description
HackaMol is an object-oriented toolkit written in Modern Perl using the Moose object system. HackaMol organizes atoms within molecules and seeks to provide chemically intuitive attributes and methods. This notebook will explore some of the features of the HackaMol::Molecule class in support of the article in review at the Journal of Chemical Information and Modeling. This notebook is available on GitHub and will evolve with the library. This notebook uses the iPerl kernel, written by Zaki Mughal, of the iPython notebook.

###Documentation for the HackaMol::Molecule Class
You can access the documentation for the HackaMol::Molecule class on [MetaCpan.org](https://metacpan.org/pod/HackaMol::Molecule). With HackaMol installed, you can load the documentation of the HackaMol::Molecule class using "perldoc HackaMol::Molecule" on the command line, see below.

###Example of using the HackaMol::Molecule class.

In this notebook we will use a common Perl library (LWP::Simple) to download a pdb from pdb.org and then use the HackaMol::Molecule class to do some calculations and manipulations.  

First load HackaMol and Modern::Perl

In [1]:
use Modern::Perl;
use HackaMol;
use Time::HiRes qw(time);

####Let's work through some tasks.

####Tasks: 
1. Download pdb from pdb.org and store it locally.  (you will need internet connection for this)
2. Load 2CBA.pdb into an instance of HackaMol::Molecule
3. Calculate the center of mass
4. Calculate the radius of gyration 
5. Translate to recenter the molecule
6. recalculate COM and Rg
7. time step 2 and steps 3-6

In [2]:
use LWP::Simple;
my $pdb = "1L2Y.pdb"; #NMR structure with several models
my $fpdb = getstore("http://pdb.org/pdb/files/$pdb","$pdb");

my $t1 = time;

my $mol = HackaMol->new
                  ->read_file_mol($pdb); # object methods chain from left to right

my $t2 = time;

say $mol->COM;
say $mol->Rg;
$mol->translate(-$mol->COM);
say $mol->COM;
say $mol->Rg;

my $t3 = time;

printf ("Time to load: %10.3fs, Time to do some calculations: %10.3f\n", $t2-$t1, $t3-$t2);

{0.43747730122783, -0.134571154696172, -0.0440167363020837}
7.29585389505559
{-4.03288862360408e-16, -7.68520023858799e-16, 6.05649433072962e-17}
7.29585389505559
Time to load:      0.196s, Time to do some calculations:      0.011


1


In [ ]:
$mol->print_pdb;

###Let's have some fun with the PDB
####Strip out the backbone and measure dihedral angles

In [ ]:
use List::Util qw (min);

my @N_CA_C = grep { $_->name eq 'N'  or 
                    $_->name eq 'CA' or 
                    $_->name eq 'C' 
             } $mol->all_atoms;

my @dihedrals = HackaMol->new
                        ->build_dihedrals(@N_CA_C);

my $maxt = min($mol->tmax,11); # we can fit up to 11 columns of dihedrals in the print out below

foreach my $dihe (@dihedrals) {
    printf( "%-22s ", $dihe->name );
    foreach my $t (0 .. $maxt){
      $dihe->gt($t);
      printf( "%7.2f ", $dihe->dihe_deg );
    }
    print "\n";
}


### Print out backbone xyz to screen

In [ ]:
HackaMol::Molecule->new (
                        atoms=>[@N_CA_C]
                    )->print_xyz;